In [1]:
"""
Test simulated outputs against real captured sensor data.
"""
# Copyright (C) 2009-2011 University of Edinburgh
#
# This file is part of IMUSim.
#
# IMUSim is free software: you can redistribute it and/or modify it
# under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# IMUSim is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with IMUSim.  If not, see <http://www.gnu.org/licenses/>.

from __future__ import division
from imusim.io.qualisys_tsv import loadQualisysTSVFile
from imusim.capture.marker import SplinedMarkerCapture
from imusim.trajectories.multi_marker import MultiMarkerTrajectory
from imusim.trajectories.offset import OffsetTrajectory
from imusim.capture.sensor import SensorDataCapture
from imusim.trajectories.rigid_body import SampledBodyModel, SampledJoint
from imusim.trajectories.rigid_body import SplinedBodyModel
from imusim.platforms.imus import IdealIMU
from imusim.behaviours.imu import BasicIMUBehaviour
from imusim.environment.base import Environment
from imusim.maths.vector_fields import NaturalNeighbourInterpolatedField
from imusim.utilities.time_series import TimeSeries
from imusim.simulation.base import Simulation
from imusim.testing.vectors import assert_vectors_correlated
import numpy as np
from os import path
import matplotlib.pyplot as plt

In [2]:
# def testAgainstReality():
print("Begin testing.")
# dir = path.dirname(__file__)
dir = path.dirname('')
# filebase = path.join(dir, "swing")
# refbase = path.join(dir, "stand")
filebase = "swing"
refbase = "stand"
# magbases = [path.join(dir, f) for f in ['magsweep1', 'magsweep2']]
magbases = [f for f in ['magsweep1', 'magsweep2']]
maglookup = {
    'Upper Leg IMU' : '66',
    'Orient 8' : '8',
    'Orient 43': '43'}
magSamples = 2000
refTime = 1.0
posStdDev = 0.0005
rotStdDev = 0.004

Begin testing.


In [3]:
ref3D = SplinedMarkerCapture(
    loadQualisysTSVFile(refbase + "_3D.tsv"), positionStdDev=posStdDev)
ref6D = SplinedMarkerCapture(
    loadQualisysTSVFile(refbase + "_6D.tsv"), rotationStdDev=rotStdDev)
capture3D = SplinedMarkerCapture(
    loadQualisysTSVFile(filebase + "_3D.tsv"), positionStdDev=posStdDev)
print("Made it 1")
captureSD = SensorDataCapture.load(filebase + ".sdc")
hip, thigh, knee, shin, ankle = \
        ['Hip', 'Thigh', 'Knee Hinge', 'Shin', 'Ankle']
jointNames = ['Upper Leg', 'Lower Leg', 'Foot']
jointAbbrevs = ['femur', 'tibia', 'foot']
orientIDs = ['66', '43', '8']
jointMarkerNames = [hip, knee, ankle]
refMarkerNames = [[thigh, knee], [shin, ankle], []]
imuMarkerNames = \
        [[j + ' IMU - ' + str(i) for i in range(1,4)] for j in jointNames]
print("imu marker names")
print(imuMarkerNames)
print("Made it 2")

Made it 1
imu marker names
[['Upper Leg IMU - 1', 'Upper Leg IMU - 2', 'Upper Leg IMU - 3'], ['Lower Leg IMU - 1', 'Lower Leg IMU - 2', 'Lower Leg IMU - 3'], ['Foot IMU - 1', 'Foot IMU - 2', 'Foot IMU - 3']]
Made it 2


In [21]:
jointMarkerSets = lambda c: [
    list(map(c.marker, jointMarkerNames)),
    [list(map(c.marker, r)) for r in refMarkerNames],
    [list(map(c.marker, i)) for i in imuMarkerNames]]
print("joint marker sets")
print(jointMarkerSets)
imuMarkerSets = lambda c: [
    [c.marker(i[0]) for i in imuMarkerNames],
    [list(map(c.marker,i[1:])) for i in imuMarkerNames]]
print("imu marker sets")
print(imuMarkerSets)
jointRefTrajectories = [MultiMarkerTrajectory(j, r + i, refTime=refTime)
    for j, r, i in zip(*(jointMarkerSets(ref3D)))]
print("j, r, i")
for j, r, i in zip(*(jointMarkerSets(ref3D))):
    print("j")
    print(j)
    print("r")
    print(r)
    print("i")
    print(i)
print(zip(*(jointMarkerSets(ref3D))))
print("joint ref trajectories")
print(jointRefTrajectories)
for trajectory in jointRefTrajectories:
    print(trajectory)
jointTrajectories = [
    MultiMarkerTrajectory(j, r + i, refVectors=m.refVectors) \
        for j, r, i, m in \
            zip(*(jointMarkerSets(capture3D) + [jointRefTrajectories]))]
for j, r, i, m in zip(*(jointMarkerSets(capture3D) + [jointRefTrajectories])):
    print("j")
    print(j)
    print("r")
    print(r)
    print("i")
    print(i)
    print("r+i")
    print(r+i)
    print("m")
    print(m)
print("joint trajectories")
print(jointTrajectories)
for trajectory in jointTrajectories:
    print(trajectory)
print("Made it 3")

joint marker sets
<function <lambda> at 0x7f091823a730>
imu marker sets
<function <lambda> at 0x7f0920b88598>
j, r, i
j
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef8ffcf8>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef9116d8>]
i
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef9119b0>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef911b00>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08eea04c88>]
j
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef911710>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef911860>]
i
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee9b10f0>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee9b1240>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee9b1390>]
j
r
[]
i
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08eea04f28>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08eea04c50>, <imusim.capture.marker.SplinedMarker3DOF

In [23]:
imuRefTrajectories = [MultiMarkerTrajectory(p, r, refTime=refTime)
    for p, r in zip(*(imuMarkerSets(ref3D)))]
for p, r in zip(*imuMarkerSets(ref3D)):
    print("p")
    print(p)
    print("r")
    print(r)
print("imu ref trajectories")
print(imuRefTrajectories)
imuVecTrajectories = [MultiMarkerTrajectory(p, r, refVectors=m.refVectors)
    for p, r, m in zip(*(imuMarkerSets(capture3D) + [imuRefTrajectories]))]
for p, r, m in zip(*(imuMarkerSets(capture3D) + [imuRefTrajectories])):
    print("p")
    print(p)
    print("r")
    print(r)
    print("m")
    print(m)
print("imu vec trajectories")
print(imuVecTrajectories)
imuRefMarkers = [ref6D.marker(j + ' IMU') for j in jointNames]
print("imu ref markers")
print(imuRefMarkers)
imuOffsets = [i.position(refTime) - j.position(refTime)
    for i, j in zip(imuRefTrajectories, jointRefTrajectories)]
for i, j in zip(imuRefTrajectories, jointRefTrajectories):
    print("i")
    print(i)
    print("j")
    print(j)
    print("i.position(refTime) - j.position(refTime)")
    print(i.position(refTime) - j.position(refTime))
print("imu offsets")
print(imuOffsets)
imuRotations = [t.rotation(refTime).conjugate * m.rotation(refTime)
    for t, m in zip(imuRefTrajectories, imuRefMarkers)]
for t, m in zip(imuRefTrajectories, imuRefMarkers):
    print("t")
    print(t)
    print("m")
    print(m)
    print("t.rotation(refTime).conjugate * m.rotation(refTime)")
    print(t.rotation(refTime).conjugate * m.rotation(refTime))
print("imu rotations")
print(imuRotations)
imuTrajectories = [OffsetTrajectory(v, o, r)
    for v, o, r in zip(imuVecTrajectories, imuOffsets, imuRotations)]
for v, o, r in zip(imuVecTrajectories, imuOffsets, imuRotations):
    print("v")
    print(v)
    print("o")
    print(o)
    print("r")
    print(r)
print("imu trajectories")
print(imuTrajectories)
print("Made it 4")

p
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ef911b00>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08eea04c88>]
p
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee9b1240>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee9b1390>]
p
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08eea04c50>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08eea04ef0>]
imu ref trajectories
[<imusim.trajectories.multi_marker.MultiMarkerTrajectory object at 0x7f08e973c780>, <imusim.trajectories.multi_marker.MultiMarkerTrajectory object at 0x7f08e973c898>, <imusim.trajectories.multi_marker.MultiMarkerTrajectory object at 0x7f08e973c908>]
p
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee7e92e8>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee7e9588>]
m
p
r
[<imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee7e9a20>, <imusim.capture.marker.SplinedMarker3DOF object at 0x7f08ee7e9b70>]
m
p
r
[<imusim.capture

In [6]:
imuData = [captureSD.device(i) for i in orientIDs]
print("imu data")
print(imuData)
joints = []
for i in range(len(jointNames)):
    name = jointNames[i]
    print("name")
    print(name)
    traj = jointTrajectories[i]
    print("traj")
    print(traj)
    if i == 0:
        model = SampledBodyModel(name)
        print("model")
        print(model)
        model.positionKeyFrames = traj.posMarker.positionKeyFrames
        print("model.positionKeyFrames")
        print(model.positionKeyFrames)
        joint = model
        print("joint")
        print(joint)
    else:
        parent = joints[-1]
        print("parent")
        print(parent)
        refTraj = jointRefTrajectories[i]
        print("refTraj")
        print(refTraj)
        parentRefTraj = jointRefTrajectories[i - 1]
        print("parentRefTraj")
        print(parentRefTraj)
        pos = refTraj.position(refTime)
        print("pos")
        print(pos)
        parentPos = parentRefTraj.position(refTime)
        print("parentPos")
        print(parentPos)
        joint = SampledJoint(joints[-1],name, offset=(pos - parentPos))
        print("joint")
        print(joint)
    joint.rotationKeyFrames = traj.rotationKeyFrames
    print("joint.rotationKeyFrames")
    print(joint.rotationKeyFrames)
    joints.append(joint)
print("Made it 5")

imu data
[<imusim.capture.sensor.CapturedDevice object at 0x7f08ee8ca208>, <imusim.capture.sensor.CapturedDevice object at 0x7f08ee8ca0f0>, <imusim.capture.sensor.CapturedDevice object at 0x7f08ee7e9f28>]
name
Upper Leg
traj
model
model.positionKeyFrames
joint
joint.rotationKeyFrames
name
Lower Leg
traj
parent
refTraj
parentRefTraj
pos
[[-0.80635   ]
 [ 0.16541209]
 [-0.51834746]]
parentPos
[[-0.80342027]
 [ 0.12976051]
 [-0.97893513]]
joint
joint.rotationKeyFrames
name
Foot
traj
parent
refTraj
parentRefTraj
pos
[[-0.86555263]
 [ 0.16672608]
 [-0.05772812]]
parentPos
[[-0.80635   ]
 [ 0.16541209]
 [-0.51834746]]
joint
joint.rotationKeyFrames
Made it 5


In [7]:
model = SplinedBodyModel(model)
print("model")
print(model)
joints = model.joints
print("joints")
print(joints)
imuJointTrajectories = [OffsetTrajectory(j, o, r)
    for j, o, r in zip(joints, imuOffsets, imuRotations)]
for j, o, r in zip(joints, imuOffsets, imuRotations):
    print("j")
    print(j)
    print("o")
    print(o)
    print("r")
    print(r)
print("imuJointTrajectories")
print(imuJointTrajectories)
positionSets = []
valueSets = []
print("Made it 6")
for magbase in magbases:
    orient = SensorDataCapture.load(magbase + '.sdc')
    print("orient")
    print(orient)
    optical = loadQualisysTSVFile(magbase + '_6D.tsv')
    print("optical")
    print(optical)
    for marker in optical.markers:
        device = orient.device(maglookup[marker.id])
        print("device")
        print(device)
        magData = device.sensorData('magnetometer').values
        print("magData")
        print(magData)
        positionSets.append(marker.positionKeyFrames.values)
        valueSets.append(
                marker.rotationKeyFrames.values.rotateVector(magData))
print("positionSets")
print(positionSets)
print("valueSets")
print(valueSets)
print("Made it 7")

model
joints
<generator object TreeNode.preorderTraversal at 0x7f08ec60a4c0>
imuJointTrajectories
[<imusim.trajectories.offset.OffsetTrajectory object at 0x7f08ee8d04e0>, <imusim.trajectories.offset.OffsetTrajectory object at 0x7f08ee8d0668>, <imusim.trajectories.offset.OffsetTrajectory object at 0x7f08ee8d0588>]
Made it 6
orient
optical
device
magData
[[        nan  0.48628733  0.49514018 ...  0.50834351  0.52766635
   0.52030011]
 [        nan -0.86247926 -0.84116914 ... -0.52743167 -0.5318245
  -0.53153775]
 [        nan  0.70581213  0.71256454 ...  0.84913463  0.84690993
   0.84469466]]
orient
optical
device
magData
[[        nan  0.04802166 -0.01190522 ... -0.20582921 -0.18388302
  -0.18194042]
 [        nan  0.27621784  0.19784042 ... -0.03798471 -0.06035326
  -0.08479882]
 [        nan  1.06467459  1.13431165 ...  0.154905    0.19468112
   0.232705  ]]
positionSets
[array([[-1.11705688, -1.11710486, -1.11709351, ..., -0.71494965,
        -0.71521948, -0.7153006 ],
       [ 0.768

In [8]:
positions = np.hstack(positionSets)
print("positions")
print(positions)
values = np.hstack(valueSets)
print("values")
print(values)
valid = ~np.any(np.isnan(positions),axis=0) & ~np.any(np.isnan(values),axis=0)
print("valid")
print(valid)
dev = values - np.median(values[:,valid],axis=1).reshape((3,1))
print("dev")
print(dev)
step = np.shape(values[:,valid])[1] // magSamples
print("step")
print(step)
posSamples = positions[:,valid][:,::step]
print("posSamples")
print(posSamples)
valSamples = values[:,valid][:,::step]
print("valSamples")
print(valSamples)
environment = Environment()
print("environment")
print(environment)
environment.magneticField = \
        NaturalNeighbourInterpolatedField(posSamples, valSamples)
print("environment.magneticField")
print(environment.magneticField)
print("Made it 8")

positions
[[-1.11705688 -1.11710486 -1.11709351 ... -0.01972801 -0.01972801
  -0.01972801]
 [ 0.76807397  0.76817291  0.76809253 ... -0.15896292 -0.15896292
  -0.15896292]
 [-0.17191766 -0.17190323 -0.1721676  ... -0.00265507 -0.00265507
  -0.00265507]]
values
[[        nan  0.27191888  0.2781566  ...  0.20340973  0.19405502
   0.20354992]
 [        nan -0.27279312 -0.25354624 ... -0.05895193 -0.02982844
  -0.00762143]
 [        nan  1.15332489  1.14840072 ...  0.15150311  0.191857
   0.23011756]]
valid
[False  True  True ...  True  True  True]
dev
[[        nan -0.03086319 -0.02462547 ... -0.09937233 -0.10872705
  -0.09923214]
 [        nan -0.26756657 -0.24831969 ... -0.05372539 -0.02460189
  -0.00239488]
 [        nan  0.16081153  0.15588737 ... -0.84101024 -0.80065635
  -0.76239579]]
step
30
posSamples
[[-1.11710486 -1.11756091 -1.1187406  ... -0.01972209 -0.01976918
  -0.01967213]
 [ 0.76817291  0.77009369  0.76874493 ... -0.15896492 -0.15896011
  -0.158989  ]
 [-0.17190323 -0.172

In [9]:
sim = Simulation(environment=environment)
print("sim")
print(sim)
sim.time = model.startTime
print("sim.time")
print(sim.time)
distortIMUs = []
dt = 1/capture3D.sampled.frameRate
print("dt")
print(dt)
for traj in imuJointTrajectories:
    platform = IdealIMU(sim, traj)
    print("platform")
    print(platform)
    distortIMUs.append(BasicIMUBehaviour(platform, dt))
print("distortIMUs")
print(distortIMUs)
print("Made it 9")

sim
sim.time
0.02734375
dt
0.00390625
platform
platform
platform
distortIMUs
[<imusim.behaviours.imu.BasicIMUBehaviour object at 0x7f08ef8edb00>, <imusim.behaviours.imu.BasicIMUBehaviour object at 0x7f08ef8ed860>, <imusim.behaviours.imu.BasicIMUBehaviour object at 0x7f08ef8ed5c0>]
Made it 9


In [10]:
sim.run(model.endTime)
i = 1
for imu in range(3):
    for sensorName in ['accelerometer', 'magnetometer', 'gyroscope']:
        sim = getattr(distortIMUs[imu].imu,sensorName).rawMeasurements
        print("sim")
        print(sim)
        print("Made it 10")
        true = imuData[imu].sensorData(sensorName)(sim.timestamps + model.startTime)
        print("true")
        print(true)
# return passed
print("Passed?")
print(assert_vectors_correlated(sim.values, true, 0.8))
        # if (sensorName == 'accelerometer'):
        #     plt.figure(i)
        #     plt.plot(true)
        #     plt.title = i
        #     plt.xlabel("Time (s)")
        #     plt.ylabel("Acceleration (m/s^2)")
        #     plt.legend()
        #     plt.show()
        #     i += 1
        # yield assert_vectors_correlated, sim.values, true, 0.8
        # return assert_vectors_correlated(sim.values, True, 0.8)

Simulating...
Simulated 0.2s of 3.4s (  5%). Estimated time remaining 5.6s
Simulated 0.3s of 3.4s ( 10%). Estimated time remaining 3.1s
Simulated 0.5s of 3.4s ( 15%). Estimated time remaining 3.8s
Simulated 0.7s of 3.4s ( 20%). Estimated time remaining 4.1s
Simulated 0.9s of 3.4s ( 25%). Estimated time remaining 3.6s
Simulated 1.0s of 3.4s ( 30%). Estimated time remaining 3.6s
Simulated 1.2s of 3.4s ( 35%). Estimated time remaining 4.6s
Simulated 1.4s of 3.4s ( 40%). Estimated time remaining 3.4s
Simulated 1.6s of 3.4s ( 45%). Estimated time remaining 2.4s
Simulated 1.7s of 3.4s ( 50%). Estimated time remaining 2.1s
Simulated 1.9s of 3.4s ( 55%). Estimated time remaining 1.5s
Simulated 2.1s of 3.4s ( 60%). Estimated time remaining 1.6s
Simulated 2.2s of 3.4s ( 65%). Estimated time remaining 1.2s
Simulated 2.4s of 3.4s ( 70%). Estimated time remaining 1.3s
Simulated 2.6s of 3.4s ( 75%). Estimated time remaining 1.0s
Simulated 2.8s of 3.4s ( 80%). Estimated time remaining 0.9s
Simulated 